# Preparing our data in Python for use in Neural Networks.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import datetime as dt
import squarify
import tensorflow as tf
from tensorflow.keras import layers
import sklearn
from sklearn.model_selection import train_test_split
import pickle
import gzip
from urllib.request import urlopen